# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
#setting 

columns = [
            'CD','total_rats', 'twentyfifth1572', 'fiftieth2517', 'eightieth3625','tons_of_refuge','tons_of_MGP', 'tons_of_paper', 'tons_res_organics', 'tons_sch_organics', 'tons_leaves_organics',
             'tons_xmastrees', 'pct_LU1', 'pct_LU2', 'pct_LU3', 'pct_LU4', 'pct_LU5', 'pct_LU6', 'pct_LU7',
             'pct_LU8', 'pct_LU9', 'pct_LU10', 'pct_LU11','BDorHigher', 'Associates', 'someCollege', 'HS',
             'LessHS', 'Adults25nOlder', 'Under18', 'eighteentoTwo'
]

target = ["fiftieth2517"]

In [4]:
# Load the data
# "mock-data" file generated from xls spreadsheets of actual data to be used after SQL db developement

df = pd.read_csv('rats_Garb_LU_ED_INC_POP_4.csv')
df.head()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
df.tail(20)

# Fill the null rows with zeros assuming particularly 
#garbage collection of organics etc is not performed and is included in refuge
df.fillna(value=0, axis=1)
df.tail(20)
df.count()

CD                      59
total_rats              59
twentyfifth1572         59
fiftieth2517            59
eightieth3625           59
tons_of_refuge          59
tons_of_MGP             59
tons_of_paper           59
tons_res_organics       59
tons_sch_organics       59
tons_leaves_organics    59
tons_xmastrees          59
pct_LU1                 59
pct_LU2                 59
pct_LU3                 59
pct_LU4                 59
pct_LU5                 59
pct_LU6                 59
pct_LU7                 59
pct_LU8                 59
pct_LU9                 59
pct_LU10                59
pct_LU11                59
BDorHigher              59
Associates              59
someCollege             59
HS                      59
LessHS                  59
Adults25nOlder          59
Under18                 59
18to24                  59
dtype: int64

In [6]:
# Convert the target column values to Below and Above based on rat data 50%tile values

x = {'Below': 'Below 50%'}   
df = df.replace(x)

x = dict.fromkeys(['Above'], 'Above 50%')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

#df.head()
df.tail(20)

,CD,total_rats,twentyfifth1572,fiftieth2517,eightieth3625,tons_of_refuge,tons_of_MGP,tons_of_paper,tons_res_organics,tons_sch_organics,...,pct_LU10,pct_LU11,BDorHigher,Associates,someCollege,HS,LessHS,Adults25nOlder,Under18,18to24
39,10 MANHATTAN,6831,Above 50%,Above 50%,Above 50%,1131518.8,54826.4,53265.6,102.9,1005.6,...,0.6,3.0,0.36,0.06,0.16,0.23,0.19,0.68,0.23,0.10
40,11 MANHATTAN,3286,Above 50%,Above 50%,Below 50%,1006505.2,44949.0,47872.4,3.8,778.3,...,1.5,4.3,0.31,0.05,0.13,0.25,0.26,0.67,0.22,0.11
41,12 MANHATTAN,4920,Above 50%,Above 50%,Above 50%,2139581.2,147001.7,112987.8,824.9,2409.6,...,1.8,6.9,0.33,0.05,0.15,0.19,0.29,0.71,0.19,0.10
42,01 QUEENS,3032,Above 50%,Above 50%,Below 50%,1744235.6,163543.1,218689.2,0.0,0.0,...,2.3,3.4,0.44,0.06,0.12,0.22,0.15,0.75,0.16,0.09
43,02 QUEENS,2348,Above 50%,Below 50%,Below 50%,1060287.7,98901.4,134628.3,2327.0,1677.4,...,5.8,4.0,0.38,0.07,0.11,0.25,0.18,0.75,0.18,0.07
44,03 QUEENS,1752,Above 50%,Below 50%,Below 50%,1808392.6,133626.9,138515.3,0.0,0.0,...,1.9,0.9,0.20,0.07,0.11,0.33,0.29,0.69,0.22,0.10
45,04 QUEENS,1641,Above 50%,Below 50%,Below 50%,1590480.8,111604.2,125139.7,0.0,0.0,...,1.3,1.3,0.22,0.07,0.13,0.30,0.29,0.69,0.22,0.09
46,05 QUEENS,4083,Above 50%,Above 50%,Above 50%,1857490.3,198092.1,261565.0,10608.9,4934.7,...,1.3,1.1,0.27,0.07,0.17,0.33,0.17,0.69,0.22,0.09
47,06 QUEENS,970,Below 50%,Below 50%,Below 50%,1094103.8,98775.5,168834.0,0.0,243.5,...,0.6,1.2,0.56,0.08,0.12,0.17,0.08,0.77,0.18,0.05
48,07 QUEENS,1752,Above 50%,Below 50%,Below 50%,2461125.3,203281.2,328594.4,6053.2,0.0,...,1.8,24.7,0.28,0.09,0.13,0.28,0.22,0.75,0.18,0.08


# Split the Data into Training and Testing

In [7]:
# Create our features
# create and drop 'fiftieth2517' column then assign to X
X = df.drop(['CD','total_rats', 'twentyfifth1572', 'fiftieth2517', 'eightieth3625'], axis=1)

# Create our target
y = df['fiftieth2517']

X.head()

,tons_of_refuge,tons_of_MGP,tons_of_paper,tons_res_organics,tons_sch_organics,tons_leaves_organics,tons_xmastrees,pct_LU1,pct_LU2,pct_LU3,...,pct_LU10,pct_LU11,BDorHigher,Associates,someCollege,HS,LessHS,Adults25nOlder,Under18,18to24
0,764007.8,32663.3,28960.3,141.5,0.0,15.6,148.7,6.1,7.9,15.2,...,4.2,4.8,0.10,0.06,0.16,0.28,0.40,0.58,0.29,0.12
1,582803.0,34424.3,24529.8,0.0,0.0,53.0,173.9,4.4,6.5,3.3,...,2.5,13.3,0.10,0.06,0.16,0.28,0.40,0.58,0.29,0.12
2,723072.8,34086.5,28189.1,0.0,0.0,16.5,91.3,10.3,12.6,13.3,...,3.5,2.2,0.12,0.06,0.18,0.28,0.36,0.56,0.30,0.13
3,1598950.4,78164.7,61554.6,343.0,0.0,0.0,269.4,4.3,12.3,15.3,...,6.3,4.2,0.14,0.07,0.17,0.28,0.35,0.60,0.29,0.11
4,1351276.7,88723.5,58439.9,21.3,0.0,11.9,202.6,11.3,16.4,18.1,...,2.1,4.7,0.12,0.07,0.17,0.29,0.35,0.58,0.30,0.12


In [8]:
X.describe()

,tons_of_refuge,tons_of_MGP,tons_of_paper,tons_res_organics,tons_sch_organics,tons_leaves_organics,tons_xmastrees,pct_LU1,pct_LU2,pct_LU3,...,pct_LU10,pct_LU11,BDorHigher,Associates,someCollege,HS,LessHS,Adults25nOlder,Under18,18to24
count,5.900000e+01,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,...,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000
mean,1.427666e+06,114866.962712,140106.747458,2149.745763,820.545763,2416.198305,724.067797,22.986441,11.245763,9.679661,...,1.694915,5.615254,0.357458,0.062203,0.141356,0.243051,0.196610,0.685932,0.217966,0.096102
std,5.254843e+05,51151.167718,77541.702580,2942.036193,1173.138766,4134.567745,605.304691,17.685416,7.658584,6.992429,...,1.334480,6.903168,0.198847,0.018761,0.037299,0.086088,0.098394,0.066491,0.056470,0.019566
min,4.017116e+05,32663.300000,24529.800000,0.000000,0.000000,0.000000,91.300000,0.100000,0.600000,0.200000,...,0.200000,0.400000,0.100000,0.020000,0.070000,0.050000,0.030000,0.550000,0.080000,0.050000
25%,1.033396e+06,80860.550000,81880.750000,0.000000,0.000000,16.050000,337.850000,5.550000,6.000000,4.350000,...,0.800000,1.300000,0.230000,0.050000,0.115000,0.205000,0.135000,0.645000,0.185000,0.090000
50%,1.351277e+06,111604.200000,132982.500000,875.000000,0.000000,480.300000,549.300000,22.300000,9.000000,8.400000,...,1.300000,3.200000,0.300000,0.060000,0.150000,0.280000,0.190000,0.680000,0.220000,0.100000
75%,1.798688e+06,142588.350000,182604.950000,3025.350000,1588.450000,3004.150000,891.850000,38.800000,15.300000,14.900000,...,2.050000,5.650000,0.415000,0.080000,0.170000,0.295000,0.265000,0.730000,0.245000,0.100000
max,2.674158e+06,227144.900000,348823.500000,10618.100000,4934.700000,19312.100000,3161.500000,59.900000,32.300000,37.100000,...,6.300000,36.400000,0.810000,0.090000,0.200000,0.440000,0.400000,0.820000,0.360000,0.160000


In [9]:
# Check the balance of our target values
y.value_counts()

Below 50%    30
Above 50%    29
Name: fiftieth2517, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [11]:
# Check balances
print(Counter(y_train))
print(Counter(y_test))

Counter({'Above 50%': 22, 'Below 50%': 22})
Counter({'Below 50%': 8, 'Above 50%': 7})


# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [12]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
# Instantiate the model
ros = RandomOverSampler(random_state=1)
# Resample the targets
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'Above 50%': 22, 'Below 50%': 22})

In [13]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='lbfgs', random_state=1)

# Fit
logreg.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [14]:
# Calculate predictions
y_pred = logreg.predict(X_test)

In [15]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.5535714285714286

In [16]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,6,1
Actual low_risk,6,2


In [17]:
# Print the imbalanced classification report
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  Above 50%       0.50      0.86      0.25      0.63      0.46      0.23         7
  Below 50%       0.67      0.25      0.86      0.36      0.46      0.20         8

avg / total       0.59      0.53      0.57      0.49      0.46      0.21        15



### SMOTE Oversampling

In [18]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
# Instantiate the model
smote = SMOTE(random_state=1)
# Resample the targets
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'Above 50%': 22, 'Below 50%': 22})

In [19]:
# Train the Logistic Regression model using the resampled data
logreg.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [20]:
# Calculated the balanced accuracy score
y_pred = logreg.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5535714285714286

In [21]:
# Display the confusion matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,6,1
Actual low_risk,6,2


In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  Above 50%       0.50      0.86      0.25      0.63      0.46      0.23         7
  Below 50%       0.67      0.25      0.86      0.36      0.46      0.20         8

avg / total       0.59      0.53      0.57      0.49      0.46      0.21        15



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [23]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
# Instantiate
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'Above 50%': 22, 'Below 50%': 22})

In [24]:
# Train the Logistic Regression model using the resampled data
logreg.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [25]:
# Calculated the balanced accuracy score
y_pred = logreg.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5535714285714286

In [26]:
# Display the confusion matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,6,1
Actual low_risk,6,2


In [27]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  Above 50%       0.50      0.86      0.25      0.63      0.46      0.23         7
  Below 50%       0.67      0.25      0.86      0.36      0.46      0.20         8

avg / total       0.59      0.53      0.57      0.49      0.46      0.21        15



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [28]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN
smoteenn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smoteenn.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'Above 50%': 4, 'Below 50%': 1})

In [29]:
# Train the Logistic Regression model using the resampled data
logreg.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [30]:
# Calculated the balanced accuracy score
y_pred = logreg.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6785714285714286

In [31]:
# Display the confusion matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,6,1
Actual low_risk,4,4


In [32]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  Above 50%       0.60      0.86      0.50      0.71      0.65      0.44         7
  Below 50%       0.80      0.50      0.86      0.62      0.65      0.41         8

avg / total       0.71      0.67      0.69      0.66      0.65      0.43        15

